# LINEAR CLASSIFICATION

## Prepare data

load data

In [1]:
import pandas as pd
import sys
sys.path.append('LINCLASS/')
import download_emb as db
import functions as f

In [2]:
train = f.load_data('/data_ssd/vera/data/twitter-english/',
                 '/data_ssd/vera/data/reddit-training-data/',
                 '/data_ssd/vera/data/train-key.json')
train['id'] = [i for i in range(len(train.index))]

test = f.load_test_data('/data_ssd/vera/data/reddit-dev-data/',
                      '/data_ssd/vera/data/dev-key.json')
test['id'] = [i for i in range(len(test.index))]

In [3]:
print(train.shape)
train.head(4)

(4878, 3)


,text,subtaskaenglish,id
521352738891132929,"What? ""@FootballcomEN: Unconfirmed reports cla...",comment,0
521356252740608001,Umdlali webhola vele RT @FootballcomEN: Unconf...,comment,1
521366121484918785,@ohlozzie @FootballcomEN Who mentioned it bein...,deny,2
521368940669583360,@AyyBant @FootballcomEN Fuck off. You're an at...,comment,3


text cleaning

In [4]:
#example
ind = 17
s = f.clean_str(train.loc[train.index[ind],'text'])
print('Before:\t', train.loc[train.index[ind],'text'])
print('After: \t', s)

Before:	 @JoeDavey7 @FootballcomEN fs Joe 😂😂😂
After: 	 fs joe 


In [5]:
train = f.clean_text(train)
test = f.clean_text(test)

create vocabulary tf-idf

In [6]:
n = 2000  #Порог частоты
voc = f.make_vocabulary(list(train.loc[:,'clean_text']), n)
tf = f.tfidf(train, voc)
tf_test = f.tfidf(test, voc)

load embeddings

In [7]:
#Скачиваем предобученные векторы,если не скачаны еще
#db.download_emb()

In [8]:
#Подгружаем эмбеддинги
emb = db.load_emb('/data_ssd/vera/data/GoogleNews-vectors-negative300.bin.gz')

loading....


twit embeddings

In [9]:
train  = f.emb_sent(train, tf, emb, dim = 300)
test = f.emb_sent(test, tf_test, emb, dim = 300)

In [10]:
#train1.to_csv('/data_ssd/vera/data/train_1.csv', sep = ';') #save

## Classification

In [11]:
#tr  = pd.read_csv('/data_ssd/vera/data/train_1.csv', sep = ';')      #если есть подготовленный

In [12]:
X = list(train.loc[:,'embed'])
y = list(train.loc[:,'subtaskaenglish'])

In [13]:
from sklearn.svm import SVC

clf = SVC(gamma='auto')
clf.fit(X, y) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

#X, y = make_classification(n_samples=1000, n_features=4,
#                            n_informative=2, n_redundant=0,
#                            random_state=0, shuffle=False)

clforest = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
clforest.fit(X, y)

#RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#            max_depth=2, max_features='auto', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#            oob_score=False, random_state=0, verbose=0, warm_start=False)
#print(clf.feature_importances_)

#print(clf.predict([[0, 0, 0, 0]]))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

## Test

In [15]:
X_test = list(train.loc[:,'embed'])
y_test = list(train.loc[:,'subtaskaenglish']) 

In [16]:
y_res = clf.predict(X_test)

In [17]:
y_res_forest = clforest.predict(X_test)

#### Metrics

In [41]:
import sklearn.metrics
acc_svc = sklearn.metrics.accuracy_score(y_test, y_res, normalize=True, sample_weight=None)
acc_forest = sklearn.metrics.accuracy_score(y_test, y_res_forest, normalize=True, sample_weight=None)
print('accuracy_score SVC:\t', acc_svc)

accuracy_score SVC:	 0.7166871668716687


In [42]:
ap_svc = sklearn.metrics.precision_score(y_test, y_res, average = 'macro')
ap_forest = sklearn.metrics.precision_score(y_test, y_res_forest, average = 'macro')
print('precision_score SVC:\t', ap_svc)

precision_score SVC:	 0.17917179171791717


In [43]:
apf_svc = sklearn.metrics.f1_score(y_test, y_res, average = 'macro')
apf_forest = sklearn.metrics.f1_score(y_test, y_res_forest, average = 'macro')
print('f1_score SVC:\t', apf_svc)

f1_score SVC:	 0.20874134224982085


# LSTM

In [40]:
sys.path.append('LSTM/')
!python LSTM/outer_semeval2019.py

Using TensorFlow backend.
Rumour Stance classification
Traceback (most recent call last):
  File "LSTM/outer_semeval2019.py", line 67, in <module>
    params, trials = parameter_search(ntrials, objective_function_stance_branchLSTM_RumEv, task)
  File "/home/verapolakova/project/RumourEval2019/baseline/LSTM/parameter_search.py", line 28, in parameter_search
    trials=trials)
  File "/data_ssd/vera/anaconda3/envs/python36_env/lib/python3.6/site-packages/hyperopt/fmin.py", line 367, in fmin
    return_argmin=return_argmin,
  File "/data_ssd/vera/anaconda3/envs/python36_env/lib/python3.6/site-packages/hyperopt/base.py", line 635, in fmin
    return_argmin=return_argmin)
  File "/data_ssd/vera/anaconda3/envs/python36_env/lib/python3.6/site-packages/hyperopt/fmin.py", line 385, in fmin
    rval.exhaust()
  File "/data_ssd/vera/anaconda3/envs/python36_env/lib/python3.6/site-packages/hyperopt/fmin.py", line 244, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchrono